# MOF Functionalizer

In [2]:
# move up a directory so PorousMaterials can find the data folder
cd("..")

using PorousMaterials

┌ Info: Precompiling PorousMaterials [68953c7c-a3c7-538e-83d3-73516288599e]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Package PorousMaterials does not have PyCall in its dependencies:
│ - If you have PorousMaterials checked out for development and have
│   added PyCall as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PorousMaterials
└ Loading PyCall into PorousMaterials from project dependency, future warnings for PorousMaterials are suppressed.


In [3]:
# move into the directory with the fragments
cd("mof_construction")

In [5]:
# # add MyModule to the path that Julia uses for modules 
push!(LOAD_PATH, joinpath("/home/ng/MOFfun.jl/src/"))

4-element Array{String,1}:
 "@"                      
 "@v#.#"                  
 "@stdlib"                
 "/home/ng/MOFfun.jl/src/"

In [6]:
using MOFun

┌ Info: Precompiling MOFun [top-level]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **



In [8]:
# The name of the fragment
# file must be a .xyz file located in ./fragments/
# The atom species that is bonded to the
# :C_aro_R atom on the ring segment of the Fragment
fragment_list = [
    "Br",
    "C-3CH3",
    "C-C",
    "CH2-CH2-CH3",
    "CH2-CH3",
    "CH2-NH2",
    "CH3",
    "CH-CH2",
    "CH-O",
    "CH-S",
    "Cl",
    "C-N",
    "F",
    "N-2CH3",
    "N-C-O",
    "NH2",
    "N-NH",
    "O-CH2-CH2-CH3",
    "O-CH2-CH3",
    "O-CH3",
    "O-C-N",
    "OH",
    "O-OH",
    "P-2CH3",
    "PH2",
    "S-CH3",
    "SH"
]

# The type of Arene Substitution ("ortho", "meta", "all") 
# TODO: "all" not yet implimented
substitution_types = ["ortho", "meta"]

2-element Array{String,1}:
 "ortho"
 "meta" 

In [9]:
# which_crystal = "NiPyC2_experiment.cif" # experimental cif
which_crystal = "NiPyC2_relax.cif" # DFT, fixed-vol relaxed cif
# which_crystal = "NiPyC2_vc-relax.cif" # DFT, vc-relaxed cif
@info which_crystal


# open crystal in lower level symmetry
# crystal = Crystal(which_crystal, convert_to_p1=true)
crystal = replicate(Crystal(which_crystal, convert_to_p1=true), (2, 1, 1))

strip_numbers_from_atom_labels!(crystal)
@debug write_xyz(crystal, joinpath("mof_construction", cryatal.name))
bonding_rules = [BondingRule(:H, :*, 0.4, 1.2),
                 BondingRule(:N, :Ni, 0.4, 2.5),
                 BondingRule(:O, :Ni, 0.4, 2.5),
                 BondingRule(:Ca, :O, 0.4, 2.5),
                 BondingRule(:Zn, :O, 0.4, 2.5),
                 BondingRule(:Br, :*, 0.4, 2.5),
                 BondingRule(:*, :*, 0.4, 1.9)]

infer_bonds!(crystal, true, bonding_rules) 
@debug write_bond_information(crystal,
    joinpath("mof_construction", remove_extension(crystal) * "_bonds.vtk"))
@debug bond_sanity_check(crystal) 

# The species of the functional atom (R) and the ipso atom of the aromatic ring
r_species = :blah
ipso_species = :blah
if which_crystal in ["NiPyC2_experiment.cif","NiPyC2_relax.cif","NiPyC2_vc-relax.cif"]
    r_species = :Ni
    ipso_species = :N
elseif which_crystal == "SBMOF-1.cif"
    r_species = :S
    ipso_species = :C
elseif which_crystal == "IRMOF-1_clean.cssr"
    r_species = :C
    ipso_species = :C
else
    error("don't know this crystal")
end

┌ Info: NiPyC2_relax.cif
└ @ Main In[9]:4


:N

In [10]:
# crystal.atoms.n
# chemical_formula(crystal)
# crystal.box.f_to_c

In [11]:
for fragment_name in fragment_list
    for sub_type in substitution_types
        if sub_type != "all"
            functionalize_mof(crystal, fragment_name, ipso_species, r_species,
                                bonding_rules; n=6, randomize_side=true,
                                arene_substitution_type=sub_type)
        elseif sub_type =="all"
            # do something special
            continue
        else
            error("substituation type not supported")
        end
        
    end
end 

Saving bond information for crystal NiPyC2_relax_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br to /home/ng/DTRA/mof_construction/NiPyC2_relax/NiPyC2_relax_ortho_functionalized_Br_bonds.vtk.
Saving bond information for crystal NiPyC2_relax_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br_NiPyC2_relax_aligned_Br to /home/ng/DTRA/mof_construction/NiPyC2_relax/NiPyC2_relax_meta_functionalized_Br_bonds.vtk.
Saving bond information for crystal NiPyC2_relax_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3_NiPyC2_relax_aligned_C-3CH3 to /home/ng/DTRA/mof_construction/NiPyC2_relax/NiPyC2_re

AssertionError: AssertionError: length(ids_C_aro) == 2

In [12]:
# functionalize_mof(xtal::Crystal, fragment_name::String, ipso_species::Symbol, r_species::Symbol,
#                                 bonding_rules::Array{BondingRule,1}; n::Int=6,
#                                 side_to_functionalize::Int=2, randomize_side::Bool=true,
#                                 arene_substitution_type::String="para")